<a href="https://colab.research.google.com/github/rdassignies/IA-Act-Chatbot/blob/main/IA_Act_instruct_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index

In [5]:
!pip install argilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.2/421.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 46.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


## Installation d'Argilla
Argilla est une plateforme open-source conçue pour faciliter la gestion, l'annotation, et la visualisation de jeux de données textuels utilisés dans le traitement du langage naturel (NLP). Elle permet aux utilisateurs de créer, modifier, et explorer des jeux de données de manière collaborative et efficace.
Elle permet notamment de créer des "feedbackdataset" particulièrement adapté pour construire des jeux de données qui nécessitent des annotations humaines par rapport aux sorties des LLMs.

In [10]:
import argilla as rg
rg.init(
            api_url=ARGILLA_API_URL,
            api_key=ARGILLA_API_KEY,
            #extra_headers = {'HF_TOKEN': HF_TOKEN}
            extra_headers={"Authorization": f"Bearer {HF_TOKEN}"}
        )

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.23.1 using a different client version (1.28.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


### Création du feedbackdataset
Le dataset comprend 2 types de champs :
- les textes qui comprennent les questions et les réponses fournies par des humains ou le modèle
- un champ permettant de noter la qualité de la réponse
- un champ permettant d'expliquer la correction


In [11]:
dataset = rg.FeedbackDataset(
            guidelines="Ce jeux de données visent à optimiser les réponses d'un modèle de langage sur l'IA Act.",
            fields=[
                rg.TextField(name="question", title="Synthetic question"),
                rg.TextField(name="human_question", title="Human question"),
                rg.TextField(name="answer", title="Generated output", use_markdown=True),
                rg.TextField(name="human_answer", title="human output", use_markdown=True)

            ],
            questions =[
                rg.RatingQuestion(
                    name="rating",
                    title="Rate the quality of the response:",
                    description="1 = very bad - 5= very good",
                    required=True,
                    values=[1,2,3,4,5]
                ),
                rg.TextQuestion(
                    name="corrected-text",
                    title="Provide a correction to the response:",
                    required=False,
                    use_markdown=True
                )
            ]
        )

In [15]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

from llama_index.llms.openai import OpenAI

import nest_asyncio
nest_asyncio.apply()

In [16]:
reader = SimpleDirectoryReader(input_dir="docs/")
documents = reader.load_data()

In [17]:
FR_QUESTION_GENERATION_PROMPT = """\
Voici les informations, tirées d'un règelement européen, dont tu disposes :
---------------------
{context_str}
---------------------
En tant que spécialiste du droit des nouvelles technologies, ton objectif est de générer 2 questions juridiques pour évaluer la compréhension du texte
ou pour susciter une compréhension plus fine du texte. Les questions doivent être strictement corrélées
avec les informations fournies. Les questions doivent être rédigées en français et formulées de manière précise et concise. Les questions
doivent également être diverses et s'adapter au contexte fourni.
{query_str}
"""
FR_ANSWER_GENERATION_PROMPT = """\
Voici les informations, tirées d'un règlement européen, dont tu disposes :
---------------------
{context_str}
---------------------
En te basant sur ces informations et sans recourir à des connaissances préalables,
génère la réponse à la question suivante :
{query_str}
"""

fr_question_generation_template = PromptTemplate(template=FR_QUESTION_GENERATION_PROMPT)
fr_answer_generation_template = PromptTemplate(template=FR_ANSWER_GENERATION_PROMPT,)

### Customisation des prompts
Llama-index fournit par défaut des prompts en anglais qu'il faut adapter. Cette partie peut être grandement affinée pour obtenir un résultat plus performant, notamment en termes de diversité dans les questions posées.

### Génération des questions/réponses synthétiques
Llama-index possède une classe prête à l'emploi pour effectuer cette tâche. Dans notre exemple, nous ne prenons que dix morceaux de texte pour générer les questions/réponses. Cependant, dans la réalité, nous prendrons l'intégralité de l'IA Act.

In [25]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1, api_key=OPENAI_API_KEY)

dataset_generator = RagDatasetGenerator.from_documents(

    documents[90:100],
    text_question_template =  fr_question_generation_template,
    text_qa_template = fr_answer_generation_template,
    #question_gen_query = fr_question_query_template,
    llm=llm,
    num_questions_per_chunk=2,  # set the number of questions per nodes
    show_progress=True,
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

### Utilisation d'un LLM pour générer des questions/réponses synthétiques
Dans notre exemple, nous utilisons un 'vieux' modèle d'OpenAI pour des raisons de coût. Il est parfaitement possible d'utiliser des modèles plus récents et d'autres fournisseurs (Mistral, Cohere, etc. )

In [26]:
rag_dataset = dataset_generator.generate_dataset_from_nodes()


100%|██████████| 10/10 [00:08<00:00,  1.21it/s]

100%|██████████| 2/2 [00:04<00:00,  2.12s/it]

100%|██████████| 2/2 [00:03<00:00,  1.81s/it]

100%|██████████| 2/2 [00:06<00:00,  3.24s/it]

100%|██████████| 2/2 [00:07<00:00,  3.56s/it]

100%|██████████| 2/2 [00:02<00:00,  1.44s/it]

100%|██████████| 2/2 [00:04<00:00,  2.10s/it]

100%|██████████| 2/2 [00:04<00:00,  2.05s/it]

100%|██████████| 2/2 [00:04<00:00,  2.23s/it]

100%|██████████| 2/2 [00:06<00:00,  3.22s/it]

100%|██████████| 2/2 [00:06<00:00,  3.17s/it]


In [41]:
df = rag_dataset.to_pandas()
df[:10][['query', 'reference_contexts', 'reference_answer']]

,query,reference_contexts,reference_answer
0,Quelles sont les obligations qui s'appliquent ...,[FRUnie dans la diversitéFRLorsque le fourniss...,Selon le règlement européen mentionné dans le ...
1,Comment les modèles d'IA à usage général prése...,[FRUnie dans la diversitéFRLorsque le fourniss...,Les modèles d'IA à usage général présentant un...
2,Quel rôle et quelle responsabilité particulier...,[FRUnie dans la diversitéFR(100) Lorsqu'un mod...,Les fournisseurs de modèles d'IA à usage génér...
3,Quelles mesures de transparence proportionnées...,[FRUnie dans la diversitéFR(100) Lorsqu'un mod...,Les fournisseurs de modèles d'IA à usage génér...
4,Quelles sont les conditions requises pour qu'u...,[FRUnie dans la diversitéFR(102) Les logiciels...,Pour qu'un modèle d'IA à usage général publié ...
5,Quels sont les critères qui déterminent si un ...,[FRUnie dans la diversitéFR(102) Les logiciels...,"Selon le règlement européen évoqué, un composa..."
6,Quelles sont les conditions sous lesquelles le...,[FRUnie dans la diversitéFR(104) Les fournisse...,Les fournisseurs de modèles d'IA à usage génér...
7,Pourquoi la publication de modèles d'IA à usag...,[FRUnie dans la diversitéFR(104) Les fournisse...,La publication de modèles d'IA à usage général...
8,Quelles sont les conditions sous lesquelles le...,[FRUnie dans la diversitéFR(105) Les modèles d...,Les fournisseurs de modèles d'IA à usage génér...
9,Quelles sont les exceptions et limitations int...,[FRUnie dans la diversitéFR(105) Les modèles d...,Les exceptions et limitations introduites par ...


In [44]:
df.iloc[1][['query', 'reference_contexts', 'reference_answer']]

query                 Comment les modèles d'IA à usage général prése...
reference_contexts    [FRUnie dans la diversitéFRLorsque le fourniss...
reference_answer      Les modèles d'IA à usage général présentant un...
Name: 1, dtype: object

### Création du dataset sur le serveur Argilla

In [48]:
for index, row in df.iterrows():
            question = row['query']
            answer = row['reference_answer']
            query_by = row['query_by']
            answer_by = row['reference_answer_by']
            record = rg.FeedbackRecord(
            fields={
                "question": question,
                'human_question' : "",
                "answer": answer,
                'human_answer' : "",

            },
            metadata= {
                'query_by' : query_by,
                 'answer_by' : answer_by
                }
            )
            dataset.add_records(record)
dataset.push_to_argilla(name="IA Act", workspace="admin")


Output()

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=3886c001-3a69-4a1a-816a-c27d87796948
   name=IA Act
   workspace=Workspace(id=99667030-16b8-43b7-8ffb-f6eb4c70e7e4, name=admin, inserted_at=2024-05-20 08:34:55.659117, updated_at=2024-05-20 08:34:55.659117)
   url=https://raphbreizh-legalgpt.hf.space/dataset/3886c001-3a69-4a1a-816a-c27d87796948/annotation-mode
   fields=[RemoteTextField(id=UUID('1e7c6e27-5374-48d5-bf30-cca08f5add89'), client=None, name='question', title='Synthetic question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('c0ef6659-483e-4b91-b1ae-11081366174d'), client=None, name='human_question', title='Human question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('bc6bbecb-a193-4954-b32d-f90428e33f78'), client=None, name='answer', title='Generated output', required=True, type='text', use_markdown=Tr

RemoteFeedbackDataset(
   id=3886c001-3a69-4a1a-816a-c27d87796948
   name=IA Act
   workspace=Workspace(id=99667030-16b8-43b7-8ffb-f6eb4c70e7e4, name=admin, inserted_at=2024-05-20 08:34:55.659117, updated_at=2024-05-20 08:34:55.659117)
   url=https://raphbreizh-legalgpt.hf.space/dataset/3886c001-3a69-4a1a-816a-c27d87796948/annotation-mode
   fields=[RemoteTextField(id=UUID('1e7c6e27-5374-48d5-bf30-cca08f5add89'), client=None, name='question', title='Synthetic question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('c0ef6659-483e-4b91-b1ae-11081366174d'), client=None, name='human_question', title='Human question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('bc6bbecb-a193-4954-b32d-f90428e33f78'), client=None, name='answer', title='Generated output', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('fda8ac35-c664-4bfa-9179-82a4d8bde3cc'), client=None, name='human_answer', title='human output', required=True,